# Template Network Definitions

<i>Version 1</i>

## References

1. ["Maintaining Knowledge about Temporal Intervals" by James F. Allen](https://cse.unl.edu/~choueiry/Documents/Allen-CACM1983.pdf) - Allen's original paper (PDF)
1. [Allen's Interval Algebra](https://www.ics.uci.edu/~alspaugh/cls/shr/allen.html) or [here](https://thomasalspaugh.org/pub/fnd/allen.html) - summarizes Allen's algebra of proper time intervals
1. [W3C Time Ontology in OWL](https://www.w3.org/TR/owl-time/) - temporal vocabulary used here is based on the W3C vocabulary of time
1. [bitsets Python package](https://bitsets.readthedocs.io/en/stable/) - used to implement Algebra relation sets and operations
1. [NetworkX Python package](http://networkx.github.io/) - used to represent directed graph of constraints
1. [Python format string syntax](https://docs.python.org/3/library/string.html#format-string-syntax) - used in Algebra summary method
1. [Spatial Ontology](https://www.w3.org/2017/sdwig/bp/) - I'm still looking for a standard spatial vocabulary; maybe start here
1. [Qualitative Spatial Relations (QSR) Library](https://qsrlib.readthedocs.io/en/latest/index.html) - an alternative library to the one defined here

## Dependencies

In [1]:
import os
import qualreas as qr
import numpy as np

In [2]:
path = os.path.join(os.getenv('PYPROJ'), 'qualreas')

In [3]:
#pt_alg = qr.Algebra(os.path.join(path, "Algebras/LinearPointAlgebra.json"))
#pt_alg = qr.Algebra(os.path.join(path, "Algebras/RightBranchingPointAlgebra.json"))
pt_alg = qr.Algebra(os.path.join(path, "Algebras/LeftBranchingPointAlgebra.json"))

## Create Algebra Elements using 4-Point Networks

### Define 4-Point Network Generator

In [4]:
def four_point_network(alg, lessthan_symbol, startname="StartPt", endname="EndPt",
                       verbose=False):
    '''Create four Temporal Entities that represent time points and use them
    to express two independent intervals. For example, (s1,e1) and (s2,e2),
    where s1 < e1 and s2 < e2, represents two proper intervals.  Using '<|='
    instead of '<', would represent two intervals where one or both might
    be points.  Return the network and the four temporal entities.'''
    net = qr.Network(alg, "Four Point Network")
    lessthan = alg.relset(lessthan_symbol)
    # Start & End Points of Interval 1
    start1 = qr.TemporalEntity(["Point"], name=startname+"1")
    end1   = qr.TemporalEntity(["Point"], name=endname+"1")
    # Start & End Points of Interval 2
    start2 = qr.TemporalEntity(["Point"], name=startname+"2")
    end2   = qr.TemporalEntity(["Point"], name=endname+"2")
    net.add_constraint(start1, end1, lessthan, verbose)
    net.add_constraint(start2, end2, lessthan, verbose)
    points = [start1, end1, start2, end2]
    return net, points

def print_4point_constraint_matrix(net, s1, e1, s2, e2):
    if not points:
        pts = [s1, e1, s2, e2]
    for a in pts:
        row = ""
        for b in pts:
            row += "  " + str(net.edges[a, b]['constraint'])
        print(row)

def constraint_matrix_to_list(net, s1, e1, s2, e2):
    pts = [s1, e1, s2, e2]
    result = []
    for a in pts:
        row = []
        for b in pts:
            row.append(str(net.edges[a, b]['constraint']))
        result.append(row)
    return result

# Map 4-Point Network "Signatures" to Typical Names
key_name_mapping = {
    '<,<,<,<': 'B', '>,>,>,>': 'BI',
    '>,<,>,<': 'D', '<,<,>,>': 'DI',
    '=,<,>,=': 'E', '=,=,=,=': 'PE',
    '>,<,>,=': 'F', '<,<,>,=': 'FI',
    '<,<,=,<': 'M', '>,=,>,>': 'MI',
    '<,<,>,<': 'O', '>,<,>,>': 'OI',
    '=,<,>,<': 'S', '=,<,>,>': 'SI',
    '>,=,>,=': 'PF', '<,<,=,=': 'PFI',
    '=,<,=,<': 'PS', '=,=,>,>': 'PSI',
    '<,<,>,r~': 'RO', '<,<,r~,r~': 'RB',
    '=,<,>,r~': 'RS', '>,<,>,r~': 'ROI',
    '>,r~,>,r~': 'RBI', 'r~,r~,r~,r~': 'R~',
    'l~,<,>,<': 'LO', 'l~,<,>,=': 'LF',
    'l~,<,>,>': 'LOI', 'l~,l~,>,>': 'LBI',
    'l~,<,l~,<': 'LB', 'l~,l~,l~,l~': 'L~'
}

### Generate a 4-Point Network

In [5]:
net4pt, s1, e1, s2, e2 = four_point_network(pt_alg, "<|=")
net4pt.summary()


Four Point Network: 4 nodes, 8 edges
  Algebra: LeftBranchingPointAlgebra
  StartPt1:
    => StartPt1: =
    => EndPt1: <|=
  EndPt1:
    => EndPt1: =
    => StartPt1: =|>
  StartPt2:
    => StartPt2: =
    => EndPt2: <|=
  EndPt2:
    => EndPt2: =
    => StartPt2: =|>


Viewing the network as a matrix, 'elem13', refers to the element in row 1, col 3, and so on for 'elem23', etc.  The matrix is 4x4, so if we partition it into four 2x2 matrices, then the two partiions on the diagonal represent two intervals and the two off-diagonal partitions represent how those two intervals relate to each other. Also, the off-diagonal 2x2 partitions are converse transposes of each other.  Oh, and the intervals represented by the diagonal partitions could be intervals, proper intervals, or points.

In [6]:
count = 0
consistent_nets = dict()
for elem13 in pt_alg.elements:
    for elem23 in pt_alg.elements:
        for elem14 in pt_alg.elements:
            for elem24 in pt_alg.elements:
                net, pt1, pt2, pt3, pt4 = four_point_network(pt_alg, "<|=")
                rs13 = pt_alg.relset(elem13)
                rs23 = pt_alg.relset(elem23)
                rs14 = pt_alg.relset(elem14)
                rs24 = pt_alg.relset(elem24)
                net.add_constraint(pt1, pt3, rs13)
                net.add_constraint(pt2, pt3, rs23)
                net.add_constraint(pt1, pt4, rs14)
                net.add_constraint(pt2, pt4, rs24)
                if net.propagate():
                    count += 1
                    elem_key = ",".join([str(rs13), str(rs14), str(rs23), str(rs24)])
                    print("==========================")
                    if elem_key in key_name_mapping:
                        print(key_name_mapping[elem_key])
                    else:
                        print("UNKNOWN")
                    #print_4point_constraint_matrix(net, pt1, pt2, pt3, pt4)
                    print(np.matrix(constraint_matrix_to_list(net, pt1, pt2, pt3, pt4)))
                    consistent_nets[key_name_mapping[elem_key]] = net
print(f"\n{count} consistent networks")

B
[['=' '<|=' '<' '<']
 ['=|>' '=' '<' '<']
 ['>' '>' '=' '<|=']
 ['>' '>' '=|>' '=']]
M
[['=' '<' '<' '<']
 ['>' '=' '=' '<']
 ['>' '=' '=' '<']
 ['>' '>' '>' '=']]
PFI
[['=' '<' '<' '<']
 ['>' '=' '=' '=']
 ['>' '=' '=' '=']
 ['>' '=' '=' '=']]
O
[['=' '<' '<' '<']
 ['>' '=' '>' '<']
 ['>' '<' '=' '<']
 ['>' '>' '>' '=']]
FI
[['=' '<' '<' '<']
 ['>' '=' '>' '=']
 ['>' '<' '=' '<']
 ['>' '=' '>' '=']]
DI
[['=' '<' '<' '<']
 ['>' '=' '>' '>']
 ['>' '<' '=' '<|=']
 ['>' '<' '=|>' '=']]
PS
[['=' '=' '=' '<']
 ['=' '=' '=' '<']
 ['=' '=' '=' '<']
 ['>' '>' '>' '=']]
PE
[['=' '=' '=' '=']
 ['=' '=' '=' '=']
 ['=' '=' '=' '=']
 ['=' '=' '=' '=']]
S
[['=' '<' '=' '<']
 ['>' '=' '>' '<']
 ['=' '<' '=' '<']
 ['>' '>' '>' '=']]
E
[['=' '<' '=' '<']
 ['>' '=' '>' '=']
 ['=' '<' '=' '<']
 ['>' '=' '>' '=']]
SI
[['=' '<' '=' '<']
 ['>' '=' '>' '>']
 ['=' '<' '=' '<']
 ['>' '<' '>' '=']]
PSI
[['=' '<' '=' '=']
 ['>' '=' '>' '>']
 ['=' '<' '=' '=']
 ['=' '<' '=' '=']]
D
[['=' '<|=' '>' '<']
 ['=|>' 

In [7]:
consistent_nets

{'B': <qualreas.Network at 0x7fa8c09e5dd8>,
 'M': <qualreas.Network at 0x7fa8a0502f60>,
 'PFI': <qualreas.Network at 0x7fa8704ccda0>,
 'O': <qualreas.Network at 0x7fa8c09dee10>,
 'FI': <qualreas.Network at 0x7fa8c09deeb8>,
 'DI': <qualreas.Network at 0x7fa8c09defd0>,
 'PS': <qualreas.Network at 0x7fa8a080ef98>,
 'PE': <qualreas.Network at 0x7fa8c09ea908>,
 'S': <qualreas.Network at 0x7fa8a080ffd0>,
 'E': <qualreas.Network at 0x7fa8c09c3f98>,
 'SI': <qualreas.Network at 0x7fa8c09de7b8>,
 'PSI': <qualreas.Network at 0x7fa8a081a860>,
 'D': <qualreas.Network at 0x7fa8a0822f98>,
 'F': <qualreas.Network at 0x7fa8a0816860>,
 'OI': <qualreas.Network at 0x7fa8a0822ef0>,
 'PF': <qualreas.Network at 0x7fa8a0824fd0>,
 'MI': <qualreas.Network at 0x7fa8a08248d0>,
 'BI': <qualreas.Network at 0x7fa8a082e860>,
 'LO': <qualreas.Network at 0x7fa8a0838ef0>,
 'LF': <qualreas.Network at 0x7fa8a08288d0>,
 'LOI': <qualreas.Network at 0x7fa8a083b4a8>,
 'LBI': <qualreas.Network at 0x7fa8a0840ba8>,
 'LB': <qualr

In [8]:
consistent_nets['B'].get_entity('StartPt2')

<TemporalEntity StartPt2 ['Point']>

In [9]:
consistent_nets['B'].get_entity('foo')

In [10]:
orig = consistent_nets['B']

In [11]:
orig.print_as_matrix()

['StartPt1', 'EndPt1', 'StartPt2', 'EndPt2']
  =  <|=  <  <
  =|>  =  <  <
  >  >  =  <|=
  >  >  =|>  =


In [12]:
orig.converse()

AttributeError: 'Network' object has no attribute 'converse'

In [ ]:
orig_nodes = list(orig.nodes())
orig_nodes

In [ ]:
pts = list(orig.edges(orig_nodes[0], orig_nodes[1]))[0]
pts

In [ ]:
pts['constraints']